# Homework 4 Supplemental Notebook

## DSC 40A, Winter 2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from itertools import combinations

### Helper Functions

Here, we'll define several functions that you'll need to use in this notebook. **Don't reinvent the wheel, use the functions that are here!**

In [2]:
def solve_normal_equations(X, y):
    '''Returns the optimal parameter vector, w*, given a design matrix X and observation vector y.'''
    return np.linalg.solve(X.T @ X, X.T @ y)

def create_design_matrix(df, columns, intercept=True):
    '''Creates a design matrix by taking the specified columns from the DataFrame df.
       Adds a column of all 1s as the first column if intercept is True, which is the default.
       The argument columns should be a list.
    '''
    df = df.copy()
    df['1'] = 1
    if intercept:
        return df[['1'] + columns].values
    else:
        return df[columns].values
    
def mean_squared_error(X, y, w):
    '''Returns the mean squared error of the predictions Xw and observations y.'''
    return np.mean((y - X @ w)**2)

## Supplement for Problem 3

Note that this question is entirely in the PDF of the homework. The code we've written here just serves to help you understand that the sum of the residuals when we have an intercept term is truly 0, using the data described below 


## Prediction problem – Billy's tips!

We will load a dataset and create two prediction rules. Run the cell below to load in a dataset containing information about the tips Billy received over the last month as a waiter at Dirty Birds.

In [3]:
tips = pd.read_csv('billy_tips.csv')
tips

,total_bill,table_size,day,tip
0,16.99,2,Sun,1.01
1,10.34,3,Sun,1.66
2,21.01,3,Sun,3.50
3,23.68,2,Sun,3.31
4,24.59,4,Sun,3.61
...,...,...,...,...
220,29.03,3,Sat,5.92
221,27.18,2,Sat,2.00
222,22.67,2,Sat,2.00
223,17.82,2,Sat,1.75


Each row corresponds to a single table that he served. Throughout this question, our goal will be to predict `tip` using some or all of the other features in the DataFrame.

Let's start by just using `total_bill` to predict `tip`. Here's a scatter plot showing the relationship between the two variables:

In [4]:
px.scatter(tips, x='total_bill', y='tip', title='Tip vs. Total Bill')

The functions defined in the **Helper Functions** section make it easy to fit a linear prediction rule:

In [5]:
X_one_feature = create_design_matrix(tips, ['total_bill'])
y = tips['tip']

# Notice that X_one_feature has two columns
X_one_feature

array([[ 1.  , 16.99],
       [ 1.  , 10.34],
       [ 1.  , 21.01],
       [ 1.  , 23.68],
       [ 1.  , 24.59],
       [ 1.  , 25.29],
       [ 1.  ,  8.77],
       [ 1.  , 26.88],
       [ 1.  , 15.04],
       [ 1.  , 14.78],
       [ 1.  , 10.27],
       [ 1.  , 35.26],
       [ 1.  , 15.42],
       [ 1.  , 18.43],
       [ 1.  , 14.83],
       [ 1.  , 21.58],
       [ 1.  , 10.33],
       [ 1.  , 16.29],
       [ 1.  , 16.97],
       [ 1.  , 20.65],
       [ 1.  , 17.92],
       [ 1.  , 20.29],
       [ 1.  , 15.77],
       [ 1.  , 39.42],
       [ 1.  , 19.82],
       [ 1.  , 17.81],
       [ 1.  , 13.37],
       [ 1.  , 12.69],
       [ 1.  , 21.7 ],
       [ 1.  , 19.65],
       [ 1.  ,  9.55],
       [ 1.  , 18.35],
       [ 1.  , 15.06],
       [ 1.  , 20.69],
       [ 1.  , 17.78],
       [ 1.  , 24.06],
       [ 1.  , 16.31],
       [ 1.  , 16.93],
       [ 1.  , 18.69],
       [ 1.  , 31.27],
       [ 1.  , 16.04],
       [ 1.  , 17.46],
       [ 1.  , 13.94],
       [ 1.

In [6]:
# Finding w*
w_one_feature = solve_normal_equations(X_one_feature, y)
w_one_feature

array([0.90415406, 0.10577454])

We can now use this prediction rule to make predictions:

In [7]:
# Dot product of an augmented feature vector for a total bill of 15 with the optimal parameter vector
np.array([1, 15]) @ w_one_feature

2.4907721362676476

In [8]:
px.scatter(tips, x='total_bill', y='tip', title='Tip vs. Total Bill')

x_range = np.linspace(0, 60)

fig = go.Figure()
fig.add_trace(go.Scatter(x = tips['total_bill'], y = y, mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = x_range, 
                         y = w_one_feature[0] + w_one_feature[1] * x_range, 
                         name = 'linear prediction rule', 
                         line=dict(color='red')))

fig.update_layout(xaxis_title = 'Total Bill', yaxis_title = 'Tip')

The mean squared error of this prediction rule is as follows:

In [9]:
mse_one_feature = mean_squared_error(X_one_feature, y, w_one_feature)
mse_one_feature

1.0892602319765285


### Predicting with two features

Now, let's suppose we want to use `total_bill` AND `table_size` to predict `tip`.




In [10]:
X_two_features = create_design_matrix(tips, ['total_bill','table_size'])
w_two_features = solve_normal_equations(X_two_features, y)
w_two_features

array([0.64108741, 0.09291565, 0.19945368])

In [11]:
mse_two_features = mean_squared_error(X_two_features, y, w_two_features)
mse_two_features

1.0653404192561737

If you completed tasks i-iii correctly, you should see a scatter plot of the original data points and your prediction rule below.

In [12]:
XX, YY = np.mgrid[0:60:2, 0:8:2]
Z = w_two_features[0] + w_two_features[1] * XX + w_two_features[2] * YY
plane = go.Surface(x=XX, y=YY, z=Z)

fig = go.Figure(data=[plane])
fig.add_trace(go.Scatter3d(x=tips['total_bill'], 
                           y=tips['table_size'], 
                           z=tips['tip'], mode='markers', marker = {'color': '#656DF1'}))

fig.update_layout(scene = dict(
    xaxis_title = 'Total Bill',
    yaxis_title = 'Table Size',
    zaxis_title = 'Tip'))

### Sum of residuals

Now let's show the sum of the residuals when we have an intercept term is truly 0


Feel free to experiment here.

In [13]:
np.sum(y - X_one_feature @ w_one_feature)

-8.881784197001252e-15

In [14]:
np.sum(y - X_two_features @ w_two_features)

-5.3290705182007514e-14